In [1]:
using CSV
using DataFrames
using Dates
using Gadfly

In [2]:
ENV["COLUMNS"] = 1000;

In [3]:
rawdata = DataFrame(CSV.File("../rawdata/reported_hospital_capacity_admissions_facility_level_weekly_average_timeseries_20210103.csv"));

In [4]:
bad_ids = begin
    id_counts = combine(groupby(rawdata, :hospital_pk), :hospital_name => (x -> length(unique(x))) => :n_names)
    filter!(x -> x.n_names > 1, id_counts)
    unique(id_counts.hospital_pk)
end
filter!(row -> !(row.hospital_pk in bad_ids), rawdata);

In [5]:
data = select(rawdata,
    :hospital_name => :hospital,
    :hospital_pk => :hospital_id,
    :collection_week => :date,
    :all_adult_hospital_inpatient_beds_7_day_avg => ByRow(x -> (ismissing(x) || x == -999999) ? missing : x) => :beds_allbeds,
    :total_icu_beds_7_day_avg => ByRow(x -> (ismissing(x) || x == -999999) ? missing : x) => :beds_icu,
);
data.beds_acute = data.beds_allbeds - data.beds_icu
sort!(data, [:hospital, :hospital_id, :date]);

In [6]:
start_date = Date(2020,  7, 1)
end_date   = Date(2020, 11, 1)
date_range = collect(start_date : Day(1) : end_date);

In [7]:
function mean_capacity(xs, dates)
    xs = [x for (x,d) in zip(xs,dates) if d in date_range]
    if all(ismissing.(xs))
        return 0
    else
        xs = filter(x -> !ismissing(x), xs)
        return round(Int, sum(xs) / length(xs))
    end
end;

function max_capacity(xs)
    if all(ismissing.(xs))
        return 0
    else
        return maximum(skipmissing(xs))
    end
end;

function min_capacity(xs)
    if all(ismissing.(xs))
        return 0
    else
        return minimum(skipmissing(xs))
    end
end;

In [8]:
capacity_data = combine(groupby(data, [:hospital, :hospital_id]), [
    :beds_icu => min_capacity => :beds_icu_min,
    :beds_acute => min_capacity => :beds_acute_min,
    :beds_allbeds => min_capacity => :beds_allbeds_min,

    [:beds_icu, :date] => mean_capacity => :beds_icu_mean,
    [:beds_acute, :date] => mean_capacity => :beds_acute_mean,
    [:beds_allbeds, :date] => mean_capacity => :beds_allbeds_mean,
        
    :beds_icu => max_capacity => :beds_icu_max,
    :beds_acute => max_capacity => :beds_acute_max,
    :beds_allbeds => max_capacity => :beds_allbeds_max,
]);

In [9]:
capacity_names_icu = [:beds_icu_min, :beds_icu_mean, :beds_icu_max];
capacity_names_acute = [:beds_acute_min, :beds_acute_mean, :beds_acute_max];
capacity_names_allbeds = [:beds_allbeds_min, :beds_allbeds_mean, :beds_allbeds_max];

In [10]:
sort!(capacity_data, [:hospital, :hospital_id]);

In [11]:
capacity_data

,hospital,hospital_id,beds_icu_min,beds_acute_min,beds_allbeds_min,beds_icu_mean,beds_acute_mean,beds_allbeds_mean,beds_icu_max,beds_acute_max,beds_allbeds_max
,String,String,Float64,Float64,Float64,Int64,Int64,Int64,Float64,Float64,Float64
1,ABBEVILLE AREA MEDICAL CENTER,421301,0.0,10.6,10.6,0,16,16,0.0,18.0,18.0
2,ABBEVILLE GENERAL HOSPITAL,190034,6.4,27.0,34.0,7,31,38,7.0,31.0,38.0
3,ABBOTT NORTHWESTERN HOSPITAL,240057,42.4,352.6,410.7,51,614,665,63.3,642.4,693.1
4,ABILENE REGIONAL MEDICAL CENTER,450558,9.3,51.4,66.0,14,68,82,21.3,83.1,104.4
5,ABINGTON MEMORIAL HOSPITAL,390231,89.0,399.0,488.0,89,432,521,90.3,437.6,527.9
6,ABRAHAM LINCOLN MEMORIAL HOSPITAL,141322,0.0,25.0,25.0,0,25,25,0.0,25.0,25.0
7,ABRAZO ARROWHEAD CAMPUS,030094,38.0,125.0,164.8,39,145,187,59.3,194.6,253.7
8,ABRAZO CENTRAL CAMPUS,030030,23.6,76.3,104.3,29,81,109,34.0,105.7,133.7
9,ABRAZO SCOTTSDALE CAMPUS,030083,12.3,37.7,51.7,14,46,60,14.3,67.9,81.9


In [12]:
capacity_data_long = stack(capacity_data, Not([:hospital, :hospital_id]), variable_name=:capacity_name, value_name=:capacity_value);

In [13]:
capacity_data_long_icu = filter(row -> Symbol(row.capacity_name) in capacity_names_icu, capacity_data_long);
capacity_data_long_acute = filter(row -> Symbol(row.capacity_name) in capacity_names_acute, capacity_data_long);
capacity_data_long_allbeds = filter(row -> Symbol(row.capacity_name) in capacity_names_allbeds, capacity_data_long);

In [14]:
capacity_data_output = select(capacity_data, :hospital, :hospital_id, :beds_icu_mean => :capacity_icu, :beds_acute_mean => :capacity_acute, :beds_allbeds_mean => :capacity_allbeds);

In [15]:
filter!(row -> row.capacity_icu + row.capacity_acute + row.capacity_allbeds > 0, capacity_data_output);

In [16]:
hospitals = collect(zip(capacity_data_output.hospital, capacity_data_output.hospital_id))
N = length(hospitals)
@show N;
@show hospitals;

N = 4787


hospitals = [("ABBEVILLE AREA MEDICAL CENTER", "421301"), ("ABBEVILLE GENERAL HOSPITAL", "190034"), ("ABBOTT NORTHWESTERN HOSPITAL", "240057"), ("ABILENE REGIONAL MEDICAL CENTER", "450558"), ("ABINGTON MEMORIAL HOSPITAL", "390231"), ("ABRAHAM LINCOLN MEMORIAL HOSPITAL", "141322"), ("ABRAZO ARROWHEAD CAMPUS", "030094"), ("ABRAZO CENTRAL CAMPUS", "030030"), ("ABRAZO SCOTTSDALE CAMPUS", "030083"), ("ABRAZO WEST CAMPUS", "030110"), ("ABROM KAPLAN MEMORIAL HOSPITAL", "191322"), ("ACADIA GENERAL HOSPITAL", "190044"), ("ACADIA ST LANDRY", "191319"), ("ACADIAN MEDICAL CENTER", "190318"), ("ACMH HOSPITAL", "390163"), ("ACUITY SPECIALTY HOSPITAL OF MORGANTOWN", "512004"), ("ACUITY SPECIALTY HOSPITAL OF NEW JERSEY", "312023"), ("ACUITY SPECIALTY HOSPITAL OF SOUTHERN NEW JERSEY", "312022"), ("ACUITY SPECIALTY HOSPITAL OHIO VALLEY", "512005"), ("AD HOSPITAL EAST, LLC", "670102"), ("ADAIR COUNTY MEMORIAL HOSPITAL", "161310"), ("ADAMS COUNTY REGIONAL MEDICAL CENTER", "361326"), ("ADAMS MEMORIAL HOSPI

 "062017"), ("NORTHERN DUTCHESS HOSPITAL", "330049"), ("NORTHERN IDAHO ADVANCED CARE HOSPITAL", "132001"), ("NORTHERN INYO HOSPITAL", "051324"), ("NORTHERN LIGHT A R GOULD HOSPITAL", "200018"), ("NORTHERN LIGHT BLUE HILL MEMORIAL HOSPITAL", "201300"), ("NORTHERN LIGHT INLAND HOSPITAL", "200041"), ("NORTHERN LIGHT MAINE COAST HOSPITAL", "200050"), ("NORTHERN LIGHT MAYO HOSPITAL", "201309"), ("NORTHERN LIGHT MERCY HOSPITAL", "200008"), ("NORTHERN LOUISIANA MEDICAL CENTER", "190086"), ("NORTHERN MAINE MEDICAL CENTER", "200052"), ("NORTHERN MONTANA HOSPITAL", "270032"), ("NORTHERN NEVADA MEDICAL CENTER", "290032"), ("NORTHERN REGIONAL HOSPITAL", "340003"), ("NORTHERN ROCKIES MEDICAL CENTER", "271337"), ("NORTHERN WESTCHESTER HOSPITAL", "330162"), ("NORTHFIELD HOSPITAL", "240014"), ("NORTHRIDGE HOSPITAL MEDICAL CENTER", "050116"), ("NORTHSHORE EXTENDED CARE HOSPITAL", "192023"), ("NORTHSHORE UNIVERSITY HEALTHSYSTEM EVANSTON HOSP", "140010"), ("NORTHSIDE HOSPITAL", "100238"), ("NORTHSIDE HOS

MAN MEDICAL CENTER HOSPITAL HILL", "260048"), ("TRUMAN MEDICAL CENTER LAKEWOOD", "260102"), ("TRUMBULL REGIONAL MEDICAL CENTER", "360055"), ("TUALITY COMMUNITY HOSPITAL", "380021"), ("TUCSON MEDICAL CENTER", "030006"), ("TUFTS MEDICAL CENTER", "220116"), ("TUG VALLEY ARH REGIONAL MEDICAL CENTER", "180069"), ("TULANE MEDICAL CENTER", "190176"), ("TULSA SPINE & SPECIALTY HOSPITAL", "370216"), ("TVRH REHABILITATION HOSPITAL", "10T290"), ("TWIN CITIES COMMUNITY HOSPITAL", "050633"), ("TWIN CITIES HOSPITAL", "100054"), ("TWIN COUNTY REGIONAL HOSPITAL", "490115"), ("TWIN LAKES REGIONAL MEDICAL CENTER", "180070"), ("TYLER CONTINUECARE HOSPITAL (AT MOTHER FRANCES)", "452091"), ("TYLER COUNTY HOSPITAL", "450460"), ("TYLER HOLMES MEMORIAL HOSPITAL CAH", "251312"), ("TYLER MEMORIAL HOSPITAL", "390192"), ("The Queen's Medical Center – West Oahu", "f2f168f72e667d52325ad37d2011f73bb21f22de60ac575efdfd5d27830b9011"), ("U P M C WESTERN MARYLAND", "210027"), ("UAMS MEDICAL CENTER", "040016"), ("UC SAN 

In [17]:
capacity_data_output |> CSV.write("../data/capacity_hhs.csv");